# Kutlwano NLP Bank Statement Model

Building a classification model for bank statement data

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData
from sklearn.model_selection import train_test_split

import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kutlw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kutlw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Connect to Database

In [2]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Kutlwano'

In [3]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [4]:
sql_query1 = "Select * from Bank_Statment"

In [5]:
df = pd.read_sql_query(sql_query1, connection)
df.head()

,Transaction Date,Posting Date,Description,Debits,Credits,Category,Column 6,CATEGORIES
0,2020/02/10,2020/02/10,MTN DATA BUNDLES,499,,COMMUNICATION,,HOME
1,2020/02/08,2020/02/09,APPLE.COM/BILL CORK IE,"74,99",,ENTERTAINMENT & EAT OUT,,FOOD
2,2020/02/07,2020/02/09,PARKHURST HARDWARE RE PARKHURST ZA,75,,HOME,,TRANSPORT
3,2020/02/07,2020/02/08,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,"31,5",,ENTERTAINMENT & EAT OUT,,LIFE AND HEALTH
4,2020/02/07,2020/02/08,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,"29,98",,ENTERTAINMENT & EAT OUT,,COMMUNICATION


## EDA

In [6]:
y = df['Category']

y.head()

0              COMMUNICATION
1    ENTERTAINMENT & EAT OUT
2                       HOME
3    ENTERTAINMENT & EAT OUT
4    ENTERTAINMENT & EAT OUT
Name: Category, dtype: object

In [7]:
df_DesValue = df.Description.values
df_DesValue1 = df.Description.values
df_DesValue[:5]

array(['MTN DATA BUNDLES', 'APPLE.COM/BILL CORK IE',
       'PARKHURST HARDWARE    RE PARKHURST ZA',
       'SEATTLE RENNIES HOUSE JOHANNESBURG ZA',
       'FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA'], dtype=object)

### Bag of Words

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=1000)
vect.fit(df['Category'])
vect.transform(df['Category']).shape

(939, 20)

In [9]:
pd.DataFrame(vect.transform(df['Category']).toarray(), columns=sorted(vect.vocabulary_.keys())).head()

,and,communication,credit,donations,eat,education,entertainment,fees,food,gifts,health,home,investment,life,other,out,saving,short,term,transport
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


### Tuning

In [10]:
vect2 = CountVectorizer(stop_words='english', 
                             max_features=500, 
                             ngram_range=(1, 1))

In [11]:
vect2.fit(df['Category'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=500, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [12]:
vect2.transform(df['Category']).shape

(939, 17)

In [13]:
pd.DataFrame(vect2.transform(df['Category']).toarray(), columns=sorted(vect2.vocabulary_.keys())).head()

,communication,credit,donations,eat,education,entertainment,fees,food,gifts,health,home,investment,life,saving,short,term,transport
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0


### Noise Removal

In [14]:
# import libraries
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# create stop words and lemmatizer instances
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

# create function to clean text
def clean_text(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    text = [lemmatizer.lemmatize(word) for word in text.split(' ')]
    text = [word for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

X = df['Description'].apply(lambda x: clean_text(x))

In [15]:
X[:]

0                             mtn data bundle
1                        applecombill cork ie
2          parkhurst hardware    parkhurst za
3       seattle rennies house johannesburg za
4      flm rennie house eater braamfontein za
                        ...                  
934                                          
935                                          
936                                          
937                                          
938                                          
Name: Description, Length: 939, dtype: object

In [16]:
vect2 = CountVectorizer()
df_Descript3 = vect2.fit_transform(X)
print(df_Descript3[0])

  (0, 420)	1
  (0, 295)	1
  (0, 232)	1


## KNN Model

### Transforming Features

In [17]:
X_transformed = pd.get_dummies(X, drop_first=True)

### Rescaling and Standardizing Features

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_transformed)
X_standardize = pd.DataFrame(X_scaled,columns=X_transformed.columns)

### Train/Test Split

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=50)

### Training

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
knn = KNeighborsClassifier()

In [21]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

### Predicting

In [22]:
pred_knn = knn.predict(X_test)

In [23]:
print(classification_report(y_test, pred_knn))

                         precision    recall  f1-score   support

                              0.21      1.00      0.35        45
          COMMUNICATION       1.00      0.59      0.74        17
ENTERTAINMENT & EAT OUT       1.00      0.19      0.32        52
                   FEES       1.00      0.86      0.93        37
                   FOOD       0.00      0.00      0.00         7
      GIFTS & DONATIONS       1.00      0.20      0.33         5
                   HOME       0.00      0.00      0.00         4
        LIFE AND HEALTH       0.00      0.00      0.00         5
                  OTHER       0.00      0.00      0.00         2
  SAVING AND INVESTMENT       0.00      0.00      0.00         5
      SHORT-TERM CREDIT       0.00      0.00      0.00         5
              TRANSPORT       1.00      0.17      0.30        98

               accuracy                           0.41       282
              macro avg       0.43      0.25      0.25       282
           weighted avg

C:\Users\kutlw\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Final Model 

In [24]:
FinalModel = pd.DataFrame({'Description' : df_DesValue1[X_train.shape[0]:],'category' : pred_knn.ravel()})

In [26]:
FinalModel.head(50)

,Description,category
0,VOD PREPAID 0825197505,
1,FEE - PRE-PAID TOP UP,
2,DR M MOKITIMI 401509249,
3,VEE 401509249,FEES
4,BUDGET2019,COMMUNICATION
5,SPOTIFYZA 5326*9249 20 DEC,FEES
6,VOD PREPAID 0825197505,
7,FEE - PRE-PAID TOP UP,
8,VOD PREPAID 0825197505,
9,FEE - PRE-PAID TOP UP,
